In [ ]:
import os
import numpy as np
import csv

Read data

In [ ]:
def data_reader(path):
    csv_reader = csv.reader(open(path, 'r'), delimiter=',')

    # Skip header
    data = []
    for i,row in enumerate(csv_reader):
        convert_list = []
        for i, element in enumerate(row):
            convert_list.append(str(element))
        data.append(convert_list)
    return data

In [ ]:
list_transactions = None
 
list_transactions = data_reader('groceries.csv')

for i in range(len(list_transactions)):
    list_transactions[i] = sorted(list_transactions[i])

Get all the distince item name

In [ ]:
list_items = None

list_items = []
for i in range(len(list_transactions)):
    list_items = list_items + list_transactions[i]
list_items = set(list_items)

list_items = sorted(list_items)

Find the trascations containt item x

In [ ]:
def Find_Itemset_Cover(X, list_transactions):
    list_transaction_cover = None
    
    list_transaction_cover = []
    X = np.array(X)
    X = np.ndarray.flatten(X)
    X = sorted(X)
    for i in range(len(list_transactions)):
        cnt = 0
        for j in range(len(X)):
            for k in range(len(list_transactions[i])):
                if (X[j] == list_transactions[i][k]):
                    cnt += 1

                if (cnt == len(X)):
                     list_transaction_cover.append(i)
                     break
        
            
            
    return list_transaction_cover

Number of transactions contains X

In [ ]:
def Cal_Absolute_Support(X, list_transactions):    
    abs_support = None
    
    abs_support = len(Find_Itemset_Cover(X,list_transactions))
    return abs_support

Percentage of transactions contains item X

In [ ]:
def Cal_Relative_Support(X, list_transactions): 
    relative_support = None
    
    Absolute = Cal_Absolute_Support(X, list_transactions)
    transaction = len(list_transactions)
    relative_support = round(Absolute / abs(transaction),4)
    return relative_support

Find all of the frequent item set containing item X

In [ ]:
def Check_Frequent_Itemset(X, list_transactions, min_sup):
    is_frequent = None
    
    Frequent = Cal_Relative_Support(X, list_transactions)
    is_frequent = (Frequent >= min_sup)
    return is_frequent

Join item sets

In [ ]:
def Check(x,y):
    if (len(x) != len(y)):
        return False

    for i in range (len(x)):
        if (x[i] != y[i]):
            return False

    return True

In [ ]:
def Cal_Self_Join(itemset_1, itemset_2):
    join_itemset = None
    
    size = min(len(itemset_1),len(itemset_2))
    cnt = 0
    
    for i in range(size):
        if (itemset_1[i] == itemset_2[i]):
            cnt += 1
    if (cnt != size - 1):
        return []

    itemset_1 = np.array(itemset_1)
    itemset_2 = np.array(itemset_2)
    join_itemset = np.concatenate((itemset_1, itemset_2))
    join_itemset = set(join_itemset)
    if (Check(join_itemset,itemset_1) or Check(join_itemset,itemset_2)):
        join_itemset = []
    return sorted(join_itemset)

Prune the item sets which not met the requirment

In [ ]:
def Prune_By_Apriori(list_candidate, list_previous_frequent_itemset):
    new_list_candidate = None

    new_list_candidate = []
    for X in list_candidate:
        cnt = 0
        for i in range(1,len(list_previous_frequent_itemset)):
            for j in range(i):
                cnt += Check(X,Cal_Self_Join(list_previous_frequent_itemset[i],list_previous_frequent_itemset[j]))
        if (cnt == len(X) - 1):
            new_list_candidate.append(X)

    return sorted(new_list_candidate)

Frequent Itemset Mining with Apriori

In [ ]:
def Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_sup):

    
    list_frequent_itemset = None
    
    N = 0
    list_frequent_itemset = [[] for i in range(500)]

    for i in range(len(list_items)):
        if (Check_Frequent_Itemset(list_items[i],list_transactions,min_sup)):
            list_frequent_itemset[N].append(list_items[i])
            N += 1

    item_size = len(list_items)
    size = 1
    start = 0
    end = N
    while (size < item_size):
          cnt = 0
          # print(1)
          # print(list_frequent_itemset[start:end])
          Current_itemsets = [[] for i in range(pow(2,N - 1))]

          for i in range(start,end):
                for j in range(start,i):
                     Current_itemsets[cnt] = (Cal_Self_Join(list_frequent_itemset[i],list_frequent_itemset[j]))
                     if (Current_itemsets[cnt] != []):
                        cnt += 1

          Unique = [0]
          Current_itemsets = sorted(Current_itemsets[0:cnt])

          for i in range(1,cnt):
            if (Check(Current_itemsets[i],Current_itemsets[i - 1]) == False):
                    Unique.append(i)

          cnt = 0
          New_Current_itemsets = [[] for i in range(len(Unique))]
          for i in Unique:
               New_Current_itemsets[cnt] = Current_itemsets[i]
               cnt += 1
          cnt -= 1

          Current_itemsets = Prune_By_Apriori(New_Current_itemsets,list_frequent_itemset[start:end])
          cnt = 0
          M = N
          

          for i in range(len(New_Current_itemsets)):
                if (Check_Frequent_Itemset(New_Current_itemsets[i],list_transactions,min_sup)):    
                   list_frequent_itemset[N] = (New_Current_itemsets[i])        
                   N += 1
          # print(2)
          # print(New_Current_itemsets)
          
          start = end
          end = N
          size += 1
          if (N == M):
             # print(N,M)
             return list_frequent_itemset[0:N]
    # print(size)
    return list_frequent_itemset[0:N]

Test on the database

In [ ]:
assert len(list_transactions) == 9835

assert list_items == sorted(list((set(list_items))))
assert len(list_items) == 169

In [ ]:
# Test on our dataset
min_support = 0.1

list_fim = Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_support)

print(list_fim)

[['bottled water'], ['other vegetables'], ['rolls/buns'], ['root vegetables'], ['soda'], ['tropical fruit'], ['whole milk'], ['yogurt']]


In [ ]:
min_support = 0.08

list_fim = Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_support)

print(list_fim)

[['bottled beer'], ['bottled water'], ['citrus fruit'], ['other vegetables'], ['pastry'], ['rolls/buns'], ['root vegetables'], ['sausage'], ['shopping bags'], ['soda'], ['tropical fruit'], ['whole milk'], ['yogurt']]
